# Bot twitter

Este bot genera reportes diarios de la calidad de aire en las últimas 24 horas y se publican en twitter.

In [1]:
# Instalar librerías
"""
!pip install APIMakeSens
!pip install folium
!pip install tweepy
!pip install selenium
"""

'\n!pip install APIMakeSens\n!pip install folium\n!pip install tweepy\n!pip install selenium\n'

### Cómo ejecutar este código automáticamente a una hora específica:

1. Descarga el notebook como un archivo .py (Asegurarse que esté en el mismo directorio que el archivo "plantilla.png".


2. Ejecuta en la terminal sudo nano /etc/crontab.


3. Añade una nueva línea al final del archivo con el siguiente formato:

    \<minuto> \<hora> * * * \<usuario> python3 /ruta/del/script.py

    \<minuto> representa el minuto de la hora en la que deseas ejecutar el comando, \<hora> representa la hora del día en formato 24 horas, y * * * significa que el comando se ejecutará cualquier día del mes y cualquier día de la semana. \<usuario> es el usuario bajo el cual se ejecutará el comando. Asegúrate de reemplazar \<usuario> con el nombre de usuario correcto.
    

4. Guarda y cierra el archivo /etc/crontab.

    Después de guardar los cambios en /etc/crontab, la tarea programada se ejecutará automáticamente según la hora especificada. Recuerda asegurarte de proporcionar la ruta correcta hacia el intérprete de Python (p.ej. /usr/bin/Python3) y la ruta correcta hacia tu script (/ruta/del/script.py). Además, ten en cuenta que el usuario especificado debe tener los permisos adecuados para ejecutar el script y acceder a cualquier archivo o recurso requerido por el script.

    
Finalmente es importante cambiar la ruta de la variable path dentro del código para que desde cron pueda acceder a ellas correctamente.

In [1]:
# Librerias
from MakeSens import MakeSens
import numpy as np
import datetime
import pytz
import folium
import pandas as pd
import os
from folium.plugins import BeautifyIcon

In [2]:
# Rango de fecha
fechaFin = datetime.datetime.now(pytz.timezone('America/Bogota')).strftime('%Y-%m-%d %H:%M:%S') # Fecha y hora Colombia
#fechaInicio = fechaFin[:-8]+f'{int(fechaFin[-8:-6])-1:02}'+fechaFin[-6:] # Restar una hora

past_date =  datetime.datetime.now(pytz.timezone('America/Bogota'))- datetime.timedelta(hours=24)  #Restar 24 horas
fechaInicio = past_date.strftime('%Y-%m-%d %H:%M:%S')

print(fechaFin,fechaInicio)

2023-06-27 08:05:26 2023-06-26 08:05:26


In [3]:
# Datos de Estaciones
Estaciones = [('E2_00023','Grupo Halley',7.1393716,-73.1210968),#7.1453716,-73.1210968
            ('mE1_00004','Col. Tecnologico',7.1381572,-73.1180935),#7.1385332,-73.115327
            ('mE1_00005','Col. Santander',7.1385332,-73.121327),#7.1381572,-73.1210935
            ('mE1_00007','Col. Normal',7.1277056,-73.116534),
            ('mE1_00006','Col. Pilar',7.108717813305484, -73.12173474310134)
            #,('mE1_00008','Col. FundeUis',7.05756, -73.0912)
             ]

In [4]:
# Creación manual de coordenadas de marcador de ubicación

ubicaciones = []
for est in Estaciones:
    ubicaciones.append([est[2],est[3]])

#shift = 0.005
shift = 0.00266
ubicaciones[0][0] += shift
ubicaciones[1][1] += shift
ubicaciones[2][1] -= shift
ubicaciones[3][0] -= shift
ubicaciones[4][0] -= shift
#ubicaciones[5][0] -= 0.00266


In [5]:
def get_pm25(Estaciones): # Obtener la media de pm25 de las estaciones
    pm25 = []
    Ntotal = int(24*30)
    for est in Estaciones:
        
        #data = MakeSens.download_data(est[0],fechaInicio,fechaFin,'m')
        file = "/".join(["../data",est[0]])
        file = ".".join([file,"csv"])
        
        if os.path.exists(file):
            data = pd.read_csv(file)
            
            if not data.empty: # Revisa que la estación tenga datos en el rango horario indicado
                data.rename(columns = {'PM2.5 1':'pm25_1'}, inplace = True)
                data.rename(columns = {'PM2.5 2':'pm25_2'}, inplace = True)
            
                data['Date'] = pd.to_datetime(data['category'])
                data.set_index('Date', inplace=True)
                data.index = data.index.strftime('%Y-%m-%d %H:%M:%S')
            
                data = data[data.index > fechaInicio]
                data = data[data.index < fechaFin]
                
                data.loc[data['pm25_1']>120,'pm25_1']=float("nan") # Eliminar valores mayores a 120
                pm25_1 = np.array(data['pm25_1'].dropna())
                data.loc[data['pm25_2']>120,'pm25_2']=float("nan") # Eliminar valores mayores a 120
                pm25_2 = np.array(data['pm25_2'].dropna())
                Ndata = data['pm25_1'].count()
                print(est[1],Ndata)
                
                if Ndata>0.5*Ntotal:     
                    pm25_mean = np.mean([data['pm25_1'].mean(),data['pm25_2'].mean()]) # Media entre la media de ambas columnas de pm25
                    pm25.append(pm25_mean)
                else:
                    pm25.append(0.0)
            else:
                pm25.append(0.0)
        else:
            pm25.append(0.0)
            
    return pm25

In [6]:
pm25 = get_pm25(Estaciones)

Grupo Halley 365
Col. Tecnologico 603
Col. Normal 694
Col. Pilar 713


### Mapa con valores de material particulado

In [7]:
# Coordenadas de Bucaramanga, Colombia

# Calcular centro entre estaciones extremas
latitudes = np.array(Estaciones)[:,2].astype('float64')
latitud = (np.max(latitudes)+np.min(latitudes))/2
longitudes = np.array(Estaciones)[:,3].astype('float64')
longitud = (np.max(longitudes)+np.min(longitudes))/2

#latitud = 7.1212675138485695
#longitud = -73.12578300721671

z = int(13-np.log2(10*(np.max(latitudes)-np.min(latitudes)))) # Zoom inicial en el mapa

# Crear el mapa de Bucaramanga con el estilo "CartoDB Positron"
bucaramanga_map = folium.Map(location=[latitud, longitud], 
                              zoom_start=z, tiles='CartoDB Positron',width=677,height=660, zoom_control=False)


def get_color(number):
    if number==0:
        return "#808080"  # Gris
    elif 0 < number <= 12:
        return '#00FF2E'  # Verde
    elif 13 <= number <= 37:
        return '#FFE000'  # Amarillo
    elif 38 <= number <= 55:
        return '#FF6100'  # Naranja
    elif 56 <= number:
        return '#FF0000'  # Rojo

    
for i in range(len(pm25)):
        
        if pm25[i] >= 0.0: # Valor crítico para alerta. Es 0 si se quieren mostrar todas las estaciones
            
            if pm25[i]==0.0:
                valor = 'X'
            else:
                valor = np.round(pm25[i],1)
            # Crear el ícono personalizado con un número
            icon_number = BeautifyIcon(
            border_color=get_color(pm25[i]),
            text_color='#000000',#get_color(pm25[i]),
            number=valor,
            background_color='transparent', # Fondo transparente para que se vean estaciones cercanas entre sí
            icon_size=(40, 40),  # Tamaño del ícono en píxeles
            inner_icon_style='line-height:25px; font-size: 12px;',  # Ajusta el tamaño de la fuente aquí
            spin=True,
            border_width=2,
            )

            # Agregar marcador al mapa
            folium.Marker(
                [Estaciones[i][2], Estaciones[i][3]],
                popup=Estaciones[i][1],
                icon=icon_number
            ).add_to(bucaramanga_map)
            
            location_marker = BeautifyIcon(
            icon_shape='square',
            text_color='#ffffff',#get_color(pm25[i]),
            number=i+1,
            border_color='#2f5786',
            background_color='#2f5786', # Fondo transparente para que se vean estaciones cercanas entre sí
            icon_size=(20, 20),  # Tamaño del ícono en píxeles
            inner_icon_style='line-height:10px; font-size: 12px;',  # Ajusta el tamaño de la fuente aquí
            spin=True,
            border_width=2,
            )

            # Agregar marcador al mapa
            folium.Marker(
                ubicaciones[i],
                popup=Estaciones[i][1],
                icon=location_marker
            ).add_to(bucaramanga_map)

#bucaramanga_map

In [8]:
bucaramanga_map

Para guardar el mapa como imagen debe tenerse la plantilla en el mismo directorio que este código

In [9]:
# Convertir mapa a imagen
#bucaramanga_map.save('mapa.html')
import io
from PIL import Image

# Ruta absoluta para acceder a archivos
path = '/home/kevin/Repos/BotAir/Bot_twitter/'

img_data = bucaramanga_map._to_png(5)
img = Image.open(io.BytesIO(img_data))

plantilla = Image.open(path+'plantilla.png') # Debe añadirse manualmente en la plantilla las estaciones a mostrar
plantilla.paste(img,(0,80)) # Se une la imagen generada con la plantilla
plantilla.save(path+'mapa_full.png')

#plantilla.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/kevin/Repos/BotAir/Bot_twitter/plantilla.png'

In [ ]:
import tweepy

# Llaves de acceso api de twitter

api_key = "HShcrrhB8RkBDziYkUhLYtuJ2"
api_key_secret = "O59rkJfdgpb9mgltXMUqG6mpZRWMY46AzVH7rskY5X1idNRWOi"
access_token= '1667571403778826240-lvL0JpYNAarZfwBbXroLpiYpc7YIYp'
access_token_secret= 'ELC2Y2jhhzzPvwVg0dPNgbfbKdMN8mSkWivQOoprkJqaA'
bearer_token='AAAAAAAAAAAAAAAAAAAAAImjoAEAAAAA9IOPR54QLozTUAmbmUGoQaIIfIw%3Df2egcbl4AtpgBM3aVttbtgggAUPmJrfLH4jZkGRqcDLx7R2uZs'

In [ ]:
# Conexión api de twitter

client = tweepy.Client(bearer_token, api_key, api_key_secret, access_token, access_token_secret)
auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
tweet = f'''#CalidadDelAire #Bucaramanga Reporte diario de calidad de aire (material particulado 2.5 mm) {fechaFin[:10]}.\n
Para conocer más sobre las estaciones y el proyecto racimo-móncora: https://class.redclara.net/halley/moncora/intro.html'''

media = api.media_upload(path+'mapa_full.png')

client.create_tweet(text = tweet, media_ids=[media.media_id_string])